# 使用LSTM與GRU進行手寫辨識預測
<br>
<img src='images/Keras sequential (LSTM GRU)模型建立.png'/>

## 目錄
- 1.導入tensorflow_keras內建的MNIST資料集
- 2.建立順序型架構模型 model
    - 加入LSTM/ GRU層
    - 依序繼續搭建整個神經網路
    - 設定最佳化器
    - 編譯模型
    - 開始訓練
    - 訓練完畢，計算訓練分數
- 3.結論

In [1]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, LSTM, SimpleRNN, GRU
import numpy as np
import matplotlib.pyplot as plt

## 1.導入tensorflow_keras內建的MNIST資料集

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 標準化數據
x_train = x_train/255.0
x_test = x_test/255.0
print(x_train.shape)
print(x_train[0].shape)

(60000, 28, 28)
(28, 28)


In [4]:
import numpy as np
np.array([[1, 2, 2]]).shape

(1, 3)

## 2.建立順序型架構模型 model

In [5]:
model = Sequential()

### 加入RNN層


In [6]:
# LSTM 模型
# model.add(LSTM(units = 128, activation='tanh', dropout=0.0,
# recurrent_dropout=0.0, return_sequences=True))

# GRU 模型
model.add(GRU(units=128, activation='tanh', dropout=0.0,
              recurrent_dropout=0.0, return_sequences=True))

#### 參數 return_sequences=True 很重要。
- return_sequences: 布林值。是返回输出序列中的最後一個輸出，還是完整的序列。(Keras官網)
- 沒有設定成True的話，第二個RNN層得到的輸入會只剩下1維。
- return_sequences會讓輸出以相當於輸入的序列傳遞到下一層。
- 如果不打算放第二層RNN就不用管這個了(defalt: False)

#### Note: 因為RNN對輸入有要求，所以如果放在第一層要確認好輸入的形式。否則第一層可以透過CNN來調整輸入資料。

### 依序繼續搭建整個神經網路



In [7]:
model.add(Dropout(0.2))

# LSTM模型
# model.add(LSTM(units=128, activation='relu'))

# GRU模型
model.add(GRU(units=128, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

### 設定最佳化器

Adam會動態調整Learning Rate，並具有慣量，讓他能夠衝破Local Minimun的侷限。

In [8]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

### 編譯模型

- Loss Function 設定成Crossentropy，並使用accuracy指標來評估訓練效果。

In [9]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt, metrics=['accuracy'])

### 開始訓練
##### Note: LSTM要訓練很久，epoch最好不要太大。

In [10]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 112s 2ms/sample - loss: 0.3771 - accuracy: 0.8802 - val_loss: 0.1094 - val_accuracy: 0.9683
Epoch 2/3
60000/60000 [==============================] - 107s 2ms/sample - loss: 0.1060 - accuracy: 0.9711 - val_loss: 0.0689 - val_accuracy: 0.9801
Epoch 3/3
60000/60000 [==============================] - 107s 2ms/sample - loss: 0.0746 - accuracy: 0.9798 - val_loss: 0.0498 - val_accuracy: 0.9860


### 訓練完畢，計算訓練分數
- 這邊是用Keras內建的evaluate計算訓練分數。
- 訓練的分數是基於model.compile時設定的metric(指標)來進行計算。
- 也可以考慮用sklearn.metrics裡面的函數來計算分數(裡面有諸多包括F1, ROC, AUC...的指標可以用來計算)。 

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)

print('測試損失度:', score[0])
print('測試準確率:', score[1])

測試損失度: 0.04979023582015652
測試準確率: 0.986


## 總結一下整個Sequential model的流程
#### - 基本上除了把SimpleRNN API 換成 LSTM/GRU API以外是沒有變的，在建置上感覺不出差異
<br>
<img src='images/LSTM GRU模型 (sequential).png'/>

## 結論
在我的測試中...
- simpleRNN(原始的RNN)的測試損失度:0.211630; 測試準確率:0.9399
- LSTM模型的測試損失度: 0.09239; 測試準確率: 0.9753 (訓練很久)
- GRU模型的測試損失度: 0.054459; 測試準確率: 0.9842 (訓練很久)

### 可以知道
- simpleRNN訓練最快，但效果最差。在Loss上的表現與LSTM與GRU可說是天差地別。
- LSTM和GRU訓練雖久，但也有與其匹配的準確度和價值。

##### Note: 事實上RNN並不擅長在圖像上做辨識，在之後的課程我們會繼續說明RNN的應用方法。
